# Creating material dictionary
- same them into csv files.

In [1]:
import numpy as np
import pandas as pd
import re
import math
import unicodedata
from lib import utility as util
from functools import partial

In [18]:
# file paths
size_file = "raw_data/サイズ表記.xlsx"
category_file = "raw_data/商品分類表_180410.xlsx"

# # import excel files
# # for size data
size_df = pd.read_excel( size_file, sheet_name='Q_最終メモ', header=0, index_col=0 )
size_df.columns = ['size','quality','brand']
size_df.index.name = 'product_id'

# for category data
category_df = pd.read_excel(category_file,  sheet_name='Sheet1', index_col=0 )
category_df.columns = ['product_name',
                       'material_type_id',
                       'material_type',
                       'manufacture_country_id',
                       'manufacture_country',
                       'category_id',
                       'category']
category_df.index.name = 'product_id'

material_df = size_df
material_df['product_name'] = category_df['product_name']

material_df.head(5)

,size,quality,brand,product_name
product_id,,,,
000001,NaN,NaN,NaN,志成貿易 輸入代行手数料
011007BE,NaN,NaN,NaN,ＢＣ イタワプレーン ダブル ２２０ｘ２７０
011007GN,NaN,NaN,NaN,ＢＣ イタワプレーン ダブル ２２０ｘ２７０
011007LB,NaN,NaN,NaN,ＢＣ イタワプレーン ダブル ２２０ｘ２７０
011007LG,NaN,NaN,NaN,ＢＣ イタワプレーン ダブル ２２０ｘ２７０


In [20]:
query_keywords = [
    'ＮＹＬＯＮ',
    'ＰＡＰＥＲ',
    'ＬＩＮＥＮ',
    'アクリル',
    'ＰＥＷＴＥＲ',
    'ＰＯＬＹＥＴＨＹＬＥＮＥ',
    'レーヨン',
    'ＦＡＫＥＬＥＡＴＨＥＲ',
    'ガルバルーフ',
    'WOOD',
    '毛',
    'ニット',
    'ＳＥＡＧＲＡＳＳ',
    'WOOL',
    'ｅｎａｍｅｌｗａｒｅ',
    'ＰＡＲＡＦＦＩＮ',
    'ＳＩＬＫ',
    'ツイストウール',
    'ＳＩＬＶＥＲ９２５',
    'ＢＲＡＳＳ',
    'ＰＥＡＲＬ',
    'ブルーミング',
    'SILK',
    'ガラス',
    'PINE',
    'ＷＯＯＬ',
    'ＧＯＬＤ ＰＬＡＴＥＤ',
    'ポリエステル',
    '麻',
    'ＣＯＴＴＯＮ',
    'フラッフィーウール',
    'ＡＣＲＹＬＩＣ',
    '綿',
    'ＰＯＬＹＥＳＴＥＲ',
    'ＩＲＯＮ',
    'ＰＡＬＭ ＷＡＸ',
    '合成皮革',
    'FUR',
    'ナイロン',
    'リサイクルレザー',
    'ＧＬＡＳＳ',
    'ＧＡＬＶＡＮＩＺＥＤ',
    'ソリッドファブリック',
    'ブリキ',
    'ＡＬＬＯＹ',
    'ＭＡＨＯＧＡＮＹ',
    'ＲＡＦＦＩＡ',
    'ＣＯＷ ＬＥＡＴＨＥＲ',
    'ＰＶＣ',
    'レジン',
    'ＴＩＮ',
    'ＷＯＯＤ',
    'ＳＴＲＡＷ',
    'ＳＴＯＮＥ',
    'ＰＬＡＳＴＩＣ',
    'ＰＡＲＡＦＩＮ',
    'ＪＵＴＥ',
    'ＦＬＡＸ',
    'ＰＯＬＹ',
    'ＡＢＡＣＡ',
    'ＢＥＥＳＷＡＸ',
    'ＥＮＡＭＥＬ',
    'ＰＵＬＰ',
    'ＳＨＥＥＰ ＬＥＡＴＨＥＲ',
    'ＭＩＲＲＯＲ',
    'ＳＴＡＩＮＬＥＳＳ',
    'ＲＥＳＩＮ',
    'ＡＲＵＲＯＧ',
    'ＣＥＲＡＭＩＣ',
    'ＡＬＵＭＩＮＩＵＭ',
    'ＣＥＤＡＲ',
    'ＴＥＲＲＡＣＯＴＴＡ',
    'ＭＩＮＫ ＦＵＲ',
    'ＳＯＲＧＨＵＭ',
    'ＡＣＡＣＩＡ',
    'ＪＡＣＫＦＵＲＵＩＴ ＬＥＡＦ',
    'ＮＡＮＧＫＡ',
    'ＣＥＬＬＵＬＯＩＤ',
    'ＬＩＬＹ',
    'ＰＯＲＣＥＬＡＩＮ',
    '花梨',
    'ＳＷＡＲＯＶＳＫＩ',
    'ＳＰＡＮＧＬＥ',
    'ＲＡＢＢＩＴ ＦＵＲ',
    'ＲＡＹＯＮ',
    'ＢＡＮＣＵＡＮ',
    'ＣＩＮＮＡＭＯＮ',
    'ＢＵＦＦＡＬＯ ＨＯＲＮ',
    'ＢＥＡＤＳ',
    'ＰＩＧ ＳＵＥＤＥ',
    'ＲＡＴＴＡＮ',
    'ＴＥＲＲＡ ＣＯＴＴＡ',
    'ＲＵＢＢＩＴ ＦＵＲ',
    'ＢＵＦＦＡＬＯ ＬＥＡＴＨＥＲ',
    'ＡＲＴＩＣＨＯＫＥ',
    'ＣＯＣＯＮＵＴ ＦＩＢＥＲ',
    'ＰＩＮＥＣＯＲＮ',
    'ＦＥＲＮ ＬＥＡＦ',
    'ＧＡＬＶＡＮＡＩＺＥＤ',
    '牛革',
    '磁器',
    'シーグラス',
    'ＣＯＰＰＥＲ',
    '桜',
    'クロームメッキ',
    'アルパカ',
    'ＣＡＬＦ ＬＥＡＴＨＥＲ',
    'ＶＯＩＬ ＣＯＴＯＮ',
    'ＨＯＬＬＹ ＬＥＡＦ',
    'ＲＥＣＹＣＬＥ ＬＥＡＴＨＥＲ',
    'ＬＡＶＥＮＤＥＲ',
    'ＳＴＥＥＬ',
    'ＦＡＫＥ ＦＵＲ',
    'ＣＯＣＯＮＵＴ',
    'ＳＨＥＬＬ',
    'ＳＥＳＢＡＮＩＡ',
    'ＣＯＷ ＳＵＥＤＥ',
    'ＰＯＮＹ',
    'ムートン',
    'ＯＲＡＮＧＥ ＦＬＯＷＥＲ',
    'ＬＥＭＯＮ ＶＥＲＢＥＮＡ',
    'ＢＡＭＢＯＯ',
    'ＢＥＴＥＬ ＮＵＴ',
    'ＳＣＲＥＷ ＰＩＮＥ',
    'ＰＩＮＥ ＣＯＮＥ',
    'ＪＡＣＫＦＵＲＵＩＴ',
    'ＰＯＷＤＥＲ',
    'ＣＨＥＳＴＮＵＴ',
    'ＲＡＴＡＮ  ＷＩＣＫＥＲ',
    'ＦＡＫＥ ＬＥＡＴＨＥＲ',
    'ＯＲＡＮＧＥ ＦＬＯＷＥＲ',
    'ＬＥＭＯＮ ＶＥＲＢＥＮＡ',
    'ＣＡＭＥＬ ＢＯＮＥ',
    'ＣＥＬＵＬＯＩＤ',
    'ＰＩＮＥ ＣＯＲＮ',
    'ＡＬＵＭＩＮＵＭ',
    '紙',
    'ＨＥＭＰ',
    'ＡＢＴ',
    'ＧＩＮＧＥＲ ＳＰＩＣＥ',
    'ＧＡＲＤＥＮＩＡ',
    'ＰＥＡＲ',
    'ＣＲＡＹ ＰＯＴ',
    'ＢＵＴＴＥＲＦＬＹ ＬＥＡＦ',
    'ＪＡＣＫ ＦＲＵＩＴ ＬＥＡＦ',
    'ＢＥＴＥＬ ＮＵＴ',
    'ＳＣＲＥＷ ＰＩＮＥ',
    'ＰＩＮＥ ＣＯＮＥ',
    'ＪＡＣＫＦＵＲＵＩＴ',
    '亜鉛',
    'リストパール',
    'アルミ',
    'ウォッシュドウール',
    'アイアン',
    'ビーズ',
    'ＭＡＮＧＯ ＬＥＡＦ',
    'アンダリ',
    'ＬＥＡＴＨＥＲ',
    'ＷＩＬＬＯＷ',
    'シェル',
    'グラシア',
    'パール',
    'リネ',
    'ブロンズ',
    'ウッド',
    'アバカ',
    'リサイクル',
    'ウール',
    'アラン',
    'ストロー',
    'オイルレザー',
    'ゴム',
    'コットン',
    'レザー',
    'ラタン',
    'アメジスト',
    '木製',
    'ＦＵＲ',
    'ホーン',
    'horn',
    '革',
    'ラフィア',
    'ポリウレタン',
    'ワッフル',
    'クリスタル',
    'ウィロー',
    'オックス',
    'キャンバス',
    'ＲＡＦＦＩＡ',
    'モヘヤ',
    'ＭＯＨＡＩＲ',
    'ボア',
    'ＰＯＬＹＵＲＥＴＨＡＮＥ',
    'ＣＡＮＶＡＳ',
    'ＤＵＲＲＹ',
    'クレープ',
    'ＳＩＬＩＣＯＮ',
    'ダイヤ',
    'ＡＣＲＹＬＩＣ',
    'ＡＭＥＴＨＹＳＴ',
    'アメジスト',
    'ＬＡＢＲＡＤＯ',
    'ＯＮＹＸ',
    'ＱＵＡＲＴＺ',
    '信楽焼',
    '富士金梅帆布',
    'ベルベット',
    'メイズ',
    'ＯＡＫ',
    'ボイル'
]

unique_qualities = util.convert_to_unique_list( util.convert_col_to_list( material_df['quality'].dropna()))
unique_product_names = util.convert_to_unique_list(util.convert_col_to_list(material_df['product_name']))                                  
unique_quality_and_product_names = (unique_qualities + unique_product_names)

print("There are {} materials".format(len(unique_quality_and_product_names)))

filtered_names = util.filter_by_another_list(unique_quality_and_product_names, query_keywords) 

print("{} query keywords have been found".format(len(query_keywords)))
print("{} of products still remain unassigned with material".format(len(filtered_names)))
filtered_names[:5]

There are 14890 materials
210 query keywords have been found
6343 of products still remain unassigned with material


['ＰＯＴＴＥＲＹ  ',
 '＊１個あたり０．０３Ｗ  ＊ＵＳＢ接続式ＡＣアダプター付＊ＩＮＰＵＴ：１００－２４０Ｖ～５０／６０Ｈｚ  ０．２Ａ ＯＵＴＰＵＴ：５Ｖ／Ａ  ＊点灯・点滅８段階調節  ',
 'ＩＮＳＩＤＥ：ＷＩＲＥ',
 '２０１７／７／２０入荷分から、ＡＴ社で発注。',
 '※ＣＯＬＯＲ ＡＳＳＯＲＴ']

In [19]:
group_dict = {
    '革': ['牛革','合成皮革','オイルレザー','レザー','leather'],
    'ウール': ['ウォッシュドウール','ウール','wool','ツイストウール','フラッフィーウール'],
    'ウッド': ['木製','ウッド','wood'],
    '毛': ['カシミア','アルパカ','アンゴラ'],
    'コットン': ['綿','コットン', 'durry','canvas','キャンバス','ベルベット','ボイル'],
    'アルミニウム': ['aluminium','アルミニウム','aluminum','アルミ'],
    'ワックス': ['wax','ワックス'],
    'ホーン': ['horn','ホーン'],
    '紙': ['paper','紙'],
    'シルバー': ['silver','シルバー'],
    'シルク': ['silk','シルク'],
    'ラフィア':['raffia','ラフィア'],
    'モヘヤ':['mohair','モヘヤ'],
    '麻': ['麻','linen','hemp','flax'],
    'ポリエステル': ['polyester','ポリエステル'],
    'ポリウレタン': ['polyurethane','ポリウレタン'],
    'アクリル': ['アクリル','acrylic'],
    'アメジスト': ['amethyst','アメジスト'],
    '藍晶石': ['labrado','藍晶石'],
    'オニキス': ['onyx','オニキス'],
    'クォーツ': ['quartz','クォーツ'],
    'セラミック': ['ceramic','セラミック'],
    '布': ['布']
}

query_keywords = sorted(query_keywords)
names = util.normalize_strings(query_keywords)
groups = util.get_groups(names,group_dict)
dictionary = { 'query_keywords': query_keywords, 'name': names, 'group': groups }
new_df = pd.DataFrame(dictionary)
new_df.head(5)

,group,name,query_keywords
0,fur,fur,FUR
1,pine,pine,PINE
2,シルク,silk,SILK
3,ウッド,wood,WOOD
4,ウール,wool,WOOL


### Export dictionaries as csv files

In [14]:
new_df.to_csv('processed_data/material.csv', header=True, mode='w')